Use pretrained model used to optimize horn antenna

   - change source from  3 x 3 hwl dipole array to horn
   - without significant altercation of the beam
  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
pip install neptune-client

In [0]:
import neptune

neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZjY2Y2M1MjktNzljYy00MTFjLWE1NTQtOTAxYTFhMmNjM2VhIn0=",
    project_qualified_name="joey0320/dipole2horn"
)

Project(joey0320/dipole2horn)

In [0]:
import sys
sys.path.append('/content/libs')

In [0]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import time
import cv2

from DataCaller import DataCaller
from EarlyStopper import *
from Net import *
from train_test import *
from utils import *

#Global variables
config = {
    'load_data':False,
    'load_test_data':True,

    'batch_size':8,
    'val_batch_size':16,
    'learning_rate':1e-4,
    'epochs':1000,
    'patience':3,

    'img_size':64,
    'img_channels':4,

    'train_label_size':6,
    #'test_label_size':3,
    'test_label_size':6,

    'output1':2,
    'output2':2,
    'result_label':4,

    'weight':torch.tensor([1.0, 1.0, 1.0, 0.1]),
}

neptune.create_experiment(name='dipole2horn', 
                          params=config, 
                          upload_source_files=['libs/DataCaller.py',
                                               'libs/EarlyStopper.py',
                                               'libs/Net.py',
                                               'libs/train_test.py',
                                               'libs/utils.py'])

https://ui.neptune.ai/joey0320/dipole2horn/e/DIP-2


Experiment(DIP-2)

In [0]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [0]:
net = Net(config['img_channels'], config['output1'], config['output2'])
net.load_state_dict(torch.load('checkpoint.pt'))

<All keys matched successfully>

In [0]:
BASE_PATH = '/content/gdrive/My Drive/Colab Notebooks/Reverse_dipole2horn/data'
TEST_DATA_PATH = os.path.join(BASE_PATH, 'test')

In [0]:
testsets = []
for f in tqdm(os.listdir(TEST_DATA_PATH)):
  path  = os.path.join(TEST_DATA_PATH, f)
  testset = DataCaller(path, config['img_size'], config['test_label_size'])
  print(testset.__len__())
  testsets.append(testset)

testset = ConcatDataset(testsets)
print(testset.__len__())  

100%|██████████| 1/1 [00:04<00:00,  4.46s/it]

200
200


In [0]:
testloader = DataLoader(testset, batch_size=1, shuffle=False)

In [0]:
net.to(device)
test(net, testloader, config['img_channels'], config['img_size'], config['result_label'], device)
neptune.log_artifact('predictions.xlsx')
#A, B, d, t

In [0]:
for num, data in enumerate(testloader):
  x, y = data
  print(y)

In [0]:
neptune.log_artifact('checkpoint.pt')

In [0]:
neptune.log_artifact('description dipole.txt')